# Twitter US Airline Sentiment Analysis :

This project focuses on extracting and engineering meaningful features from raw Twitter data to support sentiment classification (positive, negative, neutral). We will:

- Preprocess the tweet text
- Extract numeric features such as word count, punctuation, and polarity
- Use TF-IDF vectors for richer representation
- Perform Exploratory Data Analysis (EDA)
- Identify which features correlate with sentiment classes

The dataset used is the Twitter US Airline Sentiment dataset.


### Step 1: Installing Required Libraries

Before importing, we need to install the necessary libraries. These will be used for:
- Data handling (pandas, numpy)
- Text processing (spaCy, TextBlob, etc.)
- Feature engineering and visualization

The following cell installs all required packages. If you're using Jupyter Notebook, run it with a `!` before each `pip install`.


In [5]:
# Run this cell in Jupyter Notebook to install all dependencies

!pip install pandas 
!pip install numpy 
!pip install matplotlib 
!pip install seaborn 
!pip install spacy 
!pip install textblob 
!pip install scikit-learn nltk

# Download required corpora for textblob and spaCy
!python -m textblob.download_corpora
!python -m spacy download en_core_web_sm


Finished.


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\LG\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\LG\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LG\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\LG\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\LG\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\LG\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     - ------------------------------------- 0.5/12.8 MB 360.4 kB/s eta 0:00:35
     - ------------------------------------- 0.5/12.8 MB 360.4 kB/s eta 0:00:35
     - ------------------------------------- 0.5/12.8 MB 360.4 kB/s eta 0:00:35
     -- --------------------------------

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\LG\\AppData\\Local\\Temp\\pip-unpack-42rdpna6\\en_core_web_sm-3.8.0-py3-none-any.whl'
Consider using the `--user` option or check the permissions.



* now we have to import all the installed libraries 

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import spacy
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer


### Step 2: Load and Preview the Dataset

* We begin by importing the dataset using pandas.

* The dataset contains tweets related to US airlines and is labeled for sentiment analysis. We'll load it and take an initial look at its structure.


In [9]:
import pandas as pd

df = pd.read_csv("Tweets.csv")
df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'Tweets.csv'